In [4]:
import os
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [5]:
#load data
from google.colab import drive
base_path = '/content/drive'
drive.mount(base_path)

data_path = os.path.join(base_path, 'My Drive/MSIT423/HW4')
test_data = pd.read_csv(os.path.join(data_path, 'test_sample.csv'))
data = pd.read_csv(os.path.join(data_path, 'train_sample.csv'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
X_train = data.drop(columns='class', axis=1)
y_train = data['class']

X_test = test_data

In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [8]:
param = {'objective': 'binary:logistic', 'eval_metric': 'auc'}

# 5-fold cross-validation
cv_results = xgb.cv(dtrain=dtrain, params=param, nfold=5, num_boost_round=200, early_stopping_rounds=10, seed=0, metrics='auc', as_pandas=False)

In [9]:
num_boost_rounds = len(cv_results['train-auc-mean'])

bst = xgb.train(param, dtrain, num_boost_round=num_boost_rounds)

prediction = bst.predict(dtest)

In [10]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=0)

In [11]:
rf_most_important = np.argmax(rf.feature_importances_) + 1

In [12]:
np.savez('submission.npz', rf_most_important=rf_most_important, prediction=prediction)

In [13]:
npzfile = np.load('submission.npz')
print(npzfile['rf_most_important'], npzfile['prediction'].shape)

7 (500,)


In [14]:
from google.colab import files

# Download the npz file
files.download('submission.npz')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>